Openly-shared data is often published in distinct chunks, such as annual summaries of data points with each year's summary published in a separate file. In order to facilitate trend analysis on data published in that fashion, the distinct chunks can be combined.

This notebook shows an example of retrieving multiple data files, using an Application Programming Interface (API), and combining the data in those files in a single Pandas dataframe. The example shows some of the common issues encountered when using open data.

The data used in the example are available through the U.S. Department of Education's Open Data Platform, at https://data.ed.gov. That system organizes datasets in "data profiles" that describe sets of related data files.
The specific dataset used in the example is the collection of data files in the data profile with title "IDEA Section 618 Data Products: Static Tables Part B Maintenance of Effort Reduction Table 5".

Searching on that title on the Open Data Platform leads to a page which should look like the [IDEA Data Profile](IDEA-data-profile.png) image. Selecting the "Resources" link on that page presents a list of data files, each corresponding to a single school year. This example notebook shows how to retrieve all of the data files in that list and combine them in a way that enables trend analysis over all the school years for which data are available.

The Open Data Platform has an API, described in a link at the bottom right of the web application's footer. At the time of this notebook's creation, the API is documented at [CKAN version 2.9 API](http://docs.ckan.org/en/2.9/api/).

This example uses a Python module specific to the CKAN API. That Python module implements a "wrapper" around the lower-level networking modules, making it somewhat easier to use the API by abstracting some of the conventions used in the CKAN API. The Python code to use the API thus must import that CKAN API module. **NOTE:** The CKAN API module is typically not included by default in Python library sets, so it may need to be installed in order to run this notebook.

In [1]:
# Import the CKAN API wrapper module.
import ckanapi

The CKAN API documentation notes that responses to API calls are formatted as JSON documents. The json module will help display those responses.

In [2]:
import json

Make an API call to retrieve a list of data files.

The CKAN API uses the term "package" for what the Open Data Platform calls data profiles. The API funtion named "package_show" returns all the metadata describing a package. That API call needs just one parameter - a unique identifier for the package. 

The Open Data Platform lists the unique identifer for the package in this example as "cf9bff75-1577-4ca7-a957-1f0c269aeff4". (The "Show more" link may need to be selected to show the unique identifier.)

The following Python code performs the package_show API function call for the specific data profile of interest, and displays the result in a readable format.

In [3]:
# Define the parameters for the API connection.
# The required parameter for the connection is the base URL for the API functions.
# For this example, use the U.S. Department of Education's data portal at data.ed.gov.
# The CKAN API wrapper module appends any other information to the URL specific to the API version.
CKAN_BASE_URL = "https://data.ed.gov"

# Create a connection object for using the API.
remote = ckanapi.RemoteCKAN(CKAN_BASE_URL)

# Use the API call to retrieve the descriptive metadata for a "package", using the unique identifer
# in the data portal that contains them all.
IDENTIFIER = "cf9bff75-1577-4ca7-a957-1f0c269aeff4"

# Construct a dictionary object containing the parameters to use for the API call.
# For this example, the only parameter needed is the ID.
params = {'id': IDENTIFIER}

# Use the connection object to perform the API call, passing the name of the function to invoke and the parameter
# dictionary.
result = remote.call_action(action = 'package_show', data_dict = params)

# Print the response to see what the API returns. 
# Knowing it's a JSON object, format it for easier reading with the json.dumps function.
print(json.dumps(result,indent=2))

{
  "access_level": "public",
  "amended_by_user": "true",
  "author": "Office of Special Education Programs - Research to Practice Division",
  "author_email": "OSEPideadata@ed.gov",
  "bureau_code": "018:20",
  "creator_user_id": "d6645315-1307-4d3a-9295-2a4f21b3a7ac",
  "data_dictionary_pkg": "",
  "data_dictionary_pkg_format": "",
  "data_quality": false,
  "end_date": "",
  "id": "cf9bff75-1577-4ca7-a957-1f0c269aeff4",
  "indraft": "false",
  "isopen": true,
  "level_of_data": [
    "national"
  ],
  "license_id": "cc-zero",
  "license_title": "Creative Commons CCZero",
  "license_url": "http://www.opendefinition.org/licenses/cc-zero",
  "maintainer": "",
  "maintainer_email": "odp@ed.gov",
  "metadata_created": "2021-09-24T14:30:08.358273",
  "metadata_modified": "2023-03-15T13:31:15.083103",
  "name": "idea-section-618-data-products-static-tables-part-b-moe-table5",
  "notes": "IDEA Section 618 Data Products: Static Tables\r\n##Part B Maintenance of Effort Reduction and Coordina

The "resources" key contains a list of descriptive information about the data files associated with the package, similar to the partial set of key/value pairs below.
~~~
"resources": [
    {
      "access_url": "",
...
      "description": "Table 5 Number of children who received CEIS anytime in the past two years and who received special education and related services 2020-2021",
      "ed_source": "",
...
      "package_id": "cf9bff75-1577-4ca7-a957-1f0c269aeff4",
...
      "url": "https://data.ed.gov/dataset/cf9bff75-1577-4ca7-a957-1f0c269aeff4/resource/1f5632da-5044-4de6-a81c-51485434ef0c/download/2021-bmaintenancedistrict-5.xlsx",
...
    },
~~~

 The "url" key in each resources entry is a web address for where the data file is stored. Examine the URLs.

In [5]:
datafile_list = result.get('resources',None)
if datafile_list is not None:
    for datafile in datafile_list:
        url = datafile.get('url',None)
        if url is not None:
            print(url)

https://data.ed.gov/dataset/cf9bff75-1577-4ca7-a957-1f0c269aeff4/resource/1f5632da-5044-4de6-a81c-51485434ef0c/download/2021-bmaintenancedistrict-5.xlsx
https://data.ed.gov/dataset/cf9bff75-1577-4ca7-a957-1f0c269aeff4/resource/97aac191-f354-4268-9c66-f8412c1009d3/download/1920-bmaintenancedistrict-05.xlsx
https://data.ed.gov/dataset/cf9bff75-1577-4ca7-a957-1f0c269aeff4/resource/0e5691e1-7994-4c2c-8a6f-9168b0a2e02f/download/1819-bmaintenancedistrict-5.xlsx
https://data.ed.gov/dataset/cf9bff75-1577-4ca7-a957-1f0c269aeff4/resource/54df1fca-8d76-4289-b0a7-af6de4d6b6c0/download/1718-bmaintenancedistrict-5.xlsx
https://data.ed.gov/dataset/cf9bff75-1577-4ca7-a957-1f0c269aeff4/resource/d1810070-5cb3-4ff8-ac4d-d77f96648ac7/download/1617-bmaintenancedistrict-5.xlsx
https://data.ed.gov/dataset/cf9bff75-1577-4ca7-a957-1f0c269aeff4/resource/6e51e251-3f3e-4a9a-ad22-83935913b204/download/1516-bmaintenancedistrict-5.xlsx
https://data.ed.gov/dataset/cf9bff75-1577-4ca7-a957-1f0c269aeff4/resource/e6c2b63

The URLs all end in a file name with the Excel workbook file extension. Try loading them into a pandas dataframe using the read_excel function and examining the first few lines in each. Note, the URL can be used directly by read_excel.

In [63]:
import pandas as pd

len(datafile_list)

8

In [64]:
url = datafile_list[0].get('url')
df = pd.read_excel(url)
df.head(10)

,Table Identifier,bmaintenancedistrict_5,Unnamed: 2,Unnamed: 3
0,School Year,2020-21,NaN,NaN
1,Collection,Part B Maintenance of Effort Reduction and Coo...,NaN,NaN
2,Developed,2022-11-01 00:00:00,NaN,NaN
3,Revised,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,Number of children who received CEIS anytime i...,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,State,Number of reported LEAs1,Number of children who received CEIS during SY...,Number of children who received CEIS anytime d...
8,Alabama,143,+,+
9,Alaska,54,248,543


In [65]:
url = datafile_list[1].get('url')
df = pd.read_excel(url)
df.head(10)

,Table Identifier,bmaintenancedistrict_5,Unnamed: 2,Unnamed: 3
0,School Year,2019-20,NaN,NaN
1,Collection,Part B Maintenance of Effort Reduction and Coo...,NaN,NaN
2,Developed,2021-11-01 00:00:00,NaN,NaN
3,Revised,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,Number of children who received CEIS anytime i...,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,State,Number of reported LEAs1,Number of children who received CEIS during SY...,Number of children who received CEIS anytime d...
8,Alabama,142,+,+
9,Alaska,54,452,694


In [66]:
url = datafile_list[2].get('url')
df = pd.read_excel(url)
df.head(10)

,Table Identifier,bmaintenancedistrict_5,Unnamed: 2,Unnamed: 3
0,School Year,2018-19,NaN,NaN
1,Collection,Part B Maintenance of Effort Reduction and Coo...,NaN,NaN
2,Developed,2020-11-01 00:00:00,NaN,NaN
3,Revised,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,Number of children who received CEIS anytime i...,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,State,Number of reported LEAs1,Number of children who received CEIS during SY...,Number of children who received CEIS anytime d...
8,Alabama,139,+,+
9,Alaska,54,1214,868


In [71]:
url = datafile_list[3].get('url')
df = pd.read_excel(url)
df.head(10)

,Table Identifier,bmaintenancedistrict_5,Unnamed: 2,Unnamed: 3
0,School Year,2017-18,NaN,NaN
1,Collection,Part B Maintenance of Effort Reduction and Coo...,NaN,NaN
2,Developed,2019-11-01 00:00:00,NaN,NaN
3,Revised,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,Number of children who received CEIS anytime i...,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,State,Number of reported LEAs1,Number of children who received CEIS during SY...,Number of children who received CEIS anytime d...
8,Alabama,138,50,1768
9,Alaska,54,1020,586


In [68]:
url = datafile_list[5].get('url')
df = pd.read_excel(url)
df.head(10)

,Table Identifier,bmaintenancedistrict_5,Unnamed: 2,Unnamed: 3
0,School Year,2015-16,NaN,NaN
1,Collection,Part B Maintenance of Effort Reduction and Coo...,NaN,NaN
2,Developed,2017-11-01 00:00:00,NaN,NaN
3,Revised,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,Number of children who received CEIS anytime i...,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,State,Number of reported LEAs1,Number of children who received CEIS during SY...,Number of children who received CEIS anytime d...
8,Alabama,137,802,178
9,Alaska,54,747,856


In [69]:
url = datafile_list[6].get('url')
df = pd.read_excel(url)
df.head(10)

,Table Identifier,bmaintenancedistrict_5,Unnamed: 2,Unnamed: 3
0,School Year,2014-15,NaN,NaN
1,Collection,Part B Maintenance of Effort Reduction and Coo...,NaN,NaN
2,Developed,2016-11-01 00:00:00,NaN,NaN
3,Revised,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,Number of children who received CEIS anytime i...,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,State,Number of reported LEAs1,Number of children who received CEIS during SY...,Number of children who received CEIS anytime d...
8,Alabama,136,866,575
9,Alaska,54,1033,865


In [70]:
url = datafile_list[7].get('url')
df = pd.read_excel(url)
df.head(10)

,Table Identifier,bmaintenancedistrict_5,Unnamed: 2,Unnamed: 3
0,School Year,2013-14,NaN,NaN
1,Collection,Part B Maintenance of Effort Reduction and Coo...,NaN,NaN
2,Developed,2016-01-12 00:00:00,NaN,NaN
3,Revised,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,Number of children who received CEIS anytime i...,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,State,Number of reported LEAs1,Number of children who received CEIS during SY...,Number of children who received CEIS anytime d...
8,Alabama,135,2399,473
9,Alaska,54,1301,613


The URLs do load properly, but do not result in dataframes useful for analysis using the read_excel function defaults. The read_excel function defaults treat the first line as column labels, but these files contain ancillary data on the
first few lines, with the labels actually on line 8 (offset 7) and the data starting on line 9.

The read_excel function does have a parameter to specify column labels are on a specific line. Use that option to see what difference it makes on the dataframe produced. Use just the first URL for brevity.

In [73]:
url = datafile_list[0].get('url')
df = pd.read_excel(url,header=8)
df.head(10)

,State,Number of reported LEAs1,Number of children who received CEIS during SY 2020-21,"Number of children who received CEIS anytime during SY 2018-19, 2019-20, 2020-21 and received special education and related services during SY 2020-21"
0,Alabama,143.0,+,+
1,Alaska,54.0,248,543
2,American Samoa,1.0,+,+
3,Arizona,638.0,890,3730
4,Arkansas,263.0,100,388
5,Bureau of Indian Education,174.0,1126,99
6,California,1482.0,32834,29569
7,Colorado,68.0,0,29
8,Connecticut,162.0,266,33
9,Delaware,43.0,8905,1950


The resulting dataframe has better labels, but is still not ideal for combining with the other datasets. To illustrate the issue, load a second URL into another dataframe.

In [74]:
url2 = datafile_list[1].get('url')
df2 = pd.read_excel(url2,header=8)
print('Second dataset.')
df2.head(10)

Second dataset.


,State,Number of reported LEAs1,Number of children who received CEIS during SY 2019-20,"Number of children who received CEIS anytime during SY 2017-18, 2018-19, 2019-20 and received special education and related services during SY 2019-20"
0,Alabama,142.0,+,+
1,Alaska,54.0,452,694
2,American Samoa,1.0,+,+
3,Arizona,611.0,627,410
4,Arkansas,268.0,807,141
5,Bureau of Indian Education,174.0,1329,160
6,California,1402.0,42578,22469
7,Colorado,67.0,81,52
8,Connecticut,162.0,125,19
9,Delaware,42.0,12334,1445


Combine the two dataframes.

In [37]:
cdf = pd.concat([df,df2])
print('Combined dataset.')
cdf.head(10)

Combined dataset.


,State,Number of reported LEAs1,Number of children who received CEIS during SY 2020-21,"Number of children who received CEIS anytime during SY 2018-19, 2019-20, 2020-21 and received special education and related services during SY 2020-21",Number of children who received CEIS during SY 2019-20,"Number of children who received CEIS anytime during SY 2017-18, 2018-19, 2019-20 and received special education and related services during SY 2019-20"
0,Alabama,143.0,+,+,NaN,NaN
1,Alaska,54.0,248,543,NaN,NaN
2,American Samoa,1.0,+,+,NaN,NaN
3,Arizona,638.0,890,3730,NaN,NaN
4,Arkansas,263.0,100,388,NaN,NaN
5,Bureau of Indian Education,174.0,1126,99,NaN,NaN
6,California,1482.0,32834,29569,NaN,NaN
7,Colorado,68.0,0,29,NaN,NaN
8,Connecticut,162.0,266,33,NaN,NaN
9,Delaware,43.0,8905,1950,NaN,NaN


The dataframe resulting from concatenating the two datasets with default parameters retains only the LEA count from the first dataframe passed to the concat function, since the column label is "Number of LEAs reported" in both source dataframes. There are ways to avoid that with different pandas functions or concat parameters, but the result still loses the information of which school year corresponds to a given LEA count. That's because the school year is embedded in the column name for some of the data values, such as "Number of children who received CEIS during SY 2020-2021", but not the column labeled "Number of LEAs reported".

One way to solve that issue is to explicitly add the school year as another column in each dataframe. The school year is explicitly listed in row 1, column 1, but that row is skipped if the header parameter is used. So, instead of specifying a header parameter value and letting read_excel assign column labels based on the values in the specified row, use the names parameter to assign the column labels. The row containing the school year will thus still be read, and the school year can be extracted and added to the dataframe as an additional column.

In [46]:
# Read the entire spreadsheet, explicitly assigning column labels.
url1 = datafile_list[0].get('url')
df1 = pd.read_excel(url1, names=['State','Number LEAs','Number receiving CEIS',
                                 'Number receiving CEIS and special education services'])
# Extract the school year from the second row of the resulting dataframe.
school_year = df1.iloc[0].iloc[1]
print(f'School year: {school_year}')

# Add another column containing the extracted school year.
df1['School year'] = school_year

df1.head(10)

School year: 2020-21


,State,Number LEAs,Number receiving CEIS,Number receiving CEIS and special education services,School year
0,School Year,2020-21,NaN,NaN,2020-21
1,Collection,Part B Maintenance of Effort Reduction and Coo...,NaN,NaN,2020-21
2,Developed,2022-11-01 00:00:00,NaN,NaN,2020-21
3,Revised,NaN,NaN,NaN,2020-21
4,NaN,NaN,NaN,NaN,2020-21
5,Number of children who received CEIS anytime i...,NaN,NaN,NaN,2020-21
6,NaN,NaN,NaN,NaN,2020-21
7,State,Number of reported LEAs1,Number of children who received CEIS during SY...,Number of children who received CEIS anytime d...,2020-21
8,Alabama,143,+,+,2020-21
9,Alaska,54,248,543,2020-21


The first few rows are no longer useful, so they can be removed from the dataframe.

In [47]:
df1.drop(range(8),inplace=True)
df1.head(10)

,State,Number LEAs,Number receiving CEIS,Number receiving CEIS and special education services,School year
8,Alabama,143,+,+,2020-21
9,Alaska,54,248,543,2020-21
10,American Samoa,1,+,+,2020-21
11,Arizona,638,890,3730,2020-21
12,Arkansas,263,100,388,2020-21
13,Bureau of Indian Education,174,1126,99,2020-21
14,California,1482,32834,29569,2020-21
15,Colorado,68,0,29,2020-21
16,Connecticut,162,266,33,2020-21
17,Delaware,43,8905,1950,2020-21


Note the data files use a convention of storing a "+" in each cell for which data was not reported. Translate those "+" values into "not a number" (NaN) values in the dataframe to make analysis easier.

In [80]:
# Use the to_numeric function with a parameter of errors='coerce' to compensate
# for any non-numeric strings encoded in the data.
df1['Number LEAs'] = pd.to_numeric(df1['Number LEAs'], errors='coerce')
df1['Number receiving CEIS'] = pd.to_numeric(df1['Number receiving CEIS'], errors='coerce')
df1['Number receiving CEIS and special education services'] = pd.to_numeric(df1['Number receiving CEIS and special education services'], errors='coerce')
df1.head()

,State,Number LEAs,Number receiving CEIS,Number receiving CEIS and special education services,School year
8,Alabama,143.0,NaN,NaN,2020-21
9,Alaska,54.0,248.0,543.0,2020-21
10,American Samoa,1.0,NaN,NaN,2020-21
11,Arizona,638.0,890.0,3730.0,2020-21
12,Arkansas,263.0,100.0,388.0,2020-21


Perform the same operations on the second data file.

In [81]:
url2 = datafile_list[1].get('url')
df2 = pd.read_excel(url2, names=['State','Number LEAs','Number receiving CEIS',
                                 'Number receiving CEIS and special education services'])

school_year = df2.iloc[0].iloc[1]

df2['School year'] = school_year
df2.drop(range(8), inplace=True)
df2['Number LEAs'] = pd.to_numeric(df2['Number LEAs'],errors='coerce')
df2['Number receiving CEIS'] = pd.to_numeric(df2['Number receiving CEIS'],errors='coerce')
df2['Number receiving CEIS and special education services'] = pd.to_numeric(df2['Number receiving CEIS and special education services'],errors='coerce')

df2.head(10)

,State,Number LEAs,Number receiving CEIS,Number receiving CEIS and special education services,School year
8,Alabama,142.0,NaN,NaN,2019-20
9,Alaska,54.0,452.0,694.0,2019-20
10,American Samoa,1.0,NaN,NaN,2019-20
11,Arizona,611.0,627.0,410.0,2019-20
12,Arkansas,268.0,807.0,141.0,2019-20
13,Bureau of Indian Education,174.0,1329.0,160.0,2019-20
14,California,1402.0,42578.0,22469.0,2019-20
15,Colorado,67.0,81.0,52.0,2019-20
16,Connecticut,162.0,125.0,19.0,2019-20
17,Delaware,42.0,12334.0,1445.0,2019-20


Concatenate the two dataframes, explicitly resetting the index in the result.

In [82]:
cdf = pd.concat([df1,df2],ignore_index=True)

# Show all the rows in the combined dataframe for the State of Alabama.
cdf.loc[cdf['State'] == 'Alabama']

,State,Number LEAs,Number receiving CEIS,Number receiving CEIS and special education services,School year
0,Alabama,143.0,NaN,NaN,2020-21
77,Alabama,142.0,NaN,NaN,2019-20


Since all the column names match in both dataframes, but the indices in the source dataframes was ignored, the resulting dataframe has a row for each row in each source dataframe. The added "School year" column identifies which school year is applicable to each row. Performing the same operations on all the datafiles will result in a combined dataframe that can be used for trend analysis.

Write a function to perform the common operations on each datafile.

In [83]:
def prepare_datafile(url):
    df = pd.read_excel(url,names=['State','Number LEAs','Number receiving CEIS',
                                  'Number receiving CEIS and special education services'])
    school_year = df.iloc[0].iloc[1]
    df['School year'] = school_year

    df.drop(range(8),inplace=True)

    df['Number LEAs'] = pd.to_numeric(df['Number LEAs'],errors='coerce')
    df['Number receiving CEIS'] = pd.to_numeric(df['Number receiving CEIS'],errors='coerce')
    df['Number receiving CEIS and special education services'] = pd.to_numeric(df['Number receiving CEIS and special education services'],errors='coerce')

    return df

Loop over all the datafiles, preparing them using the above function, and combining the result.

In [84]:
cdf = None
for datafile in datafile_list:
    url = datafile.get('url',None)
    if url is not None:
        # Ensure the url actually looks like an Excel spreadsheet.
        if url.endswith('.xlsx'):
            df = prepare_datafile(url)
            if cdf is None:
                cdf = df
            else:
                cdf = pd.concat([cdf,df],ignore_index=True)

# Check all the values for the State of Alabama.
cdf.loc[cdf['State'] == 'Alabama']

,State,Number LEAs,Number receiving CEIS,Number receiving CEIS and special education services,School year
0,Alabama,143.0,NaN,NaN,2020-21
77,Alabama,142.0,NaN,NaN,2019-20
154,Alabama,139.0,NaN,NaN,2018-19
229,Alabama,138.0,50.0,1768.0,2017-18
304,Alabama,137.0,933.0,149.0,2016-17
379,Alabama,137.0,802.0,178.0,2015-16
454,Alabama,136.0,866.0,575.0,2014-15
529,Alabama,135.0,2399.0,473.0,2013-14


To illustrate the type of analysis that can be done on the combined dataset, aggregate the child counts by school year.

In [91]:
gdf = cdf.groupby(['School year']).agg({'Number receiving CEIS':'sum',
                                        'Number receiving CEIS and special education services':'sum'})
gdf

,Number receiving CEIS,Number receiving CEIS and special education services
School year,,
2013-14,1577168.0,239170.0
2014-15,1470812.0,277636.0
2015-16,1412122.0,230986.0
2016-17,1534814.0,248942.0
2017-18,1311272.0,230268.0
2018-19,859082.0,131960.0
2019-20,803532.0,181508.0
2020-21,629262.0,160156.0
